In [1]:
from ae_feature_extraction import *

In [5]:
train_label=pd.read_csv(DATA_PATH +"TD_mode.csv", header=None).values

In [180]:
class AircraftDataset(Dataset):
    def __init__(self, df, labels):# df is a dataframe and label is an array indicate the true failure mode
        self.df = df.groupby("Unit").agg(list).reset_index()
        self.labels=labels

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        data = {}
#         sensor = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30',
#                   'phi', 'NRf', 'NRc', 'BPR', 'htBleed', 'W31', 'W32']
        sensor=['T24','T30','T50','P30','Ps30','phi']
        multi_sensor = []
        for sensor_name in sensor:
            multi_sensor.append(np.array(self.df[sensor_name].values.tolist()[idx]))
            single_sensor = np.array(self.df[sensor_name].values.tolist()[idx])[:, None]
            #data[sensor_name] = torch.tensor(single_sensor, dtype=torch.float)
        multi_sensor = np.vstack(multi_sensor).transpose(1, 0)
        data["input"] = torch.tensor(multi_sensor, dtype=torch.float)
        data["lifetime"] = torch.tensor(len(multi_sensor), dtype=torch.int64)
        #data["timeseries"] = torch.tensor(np.array(self.df["Time"].values.tolist()[idx])[:, None], dtype=torch.int64)
        if self.labels[idx].item()==-1:
            data["mode"]=torch.tensor([1,0],dtype=torch.float)
        elif self.labels[idx].item()==1:
            data["mode"]=torch.tensor([0,1],dtype=torch.float)
        return data

In [181]:
train_dataset=AircraftDataset(df_train,train_label)

In [182]:
train_dataset[0]

{'input': tensor([[ 0.9361,  0.3063,  0.2995, -0.7938,  0.5187, -0.2241],
         [ 1.0359,  0.4529,  0.3033, -0.4428,  0.3692, -0.0984],
         [ 0.7912,  0.2064,  0.8094, -0.5262,  0.3459, -0.4868],
         ...,
         [ 1.5619,  1.3918,  1.4484, -1.6229,  1.5142, -1.5564],
         [ 1.6165,  1.5726,  1.5852, -1.5222,  1.4769, -1.5536],
         [ 1.7176,  1.3226,  1.6458, -1.5866,  1.6381, -1.6301]]),
 'lifetime': tensor(259),
 'mode': tensor([1., 0.])}

In [183]:
class AircraftDataset_expend(AircraftDataset):  # 截断原有的数据集，获得海量的数据
    def __init__(self, df,labels,add_zero):
        super().__init__(df,labels)
        self.add_zero = add_zero
        self.cut_data()

    def cut_data(self):
        lenth = super().__len__()
        input_signal = []
        RUL = []
        label=[]
        for unit in range(lenth):
            unit_input = super().__getitem__(unit)["input"]
            unit_label=super().__getitem__(unit)["mode"]
            unit_life = len(unit_input)
            if self.add_zero:
                for time in range(3, unit_life):
                    input_tensor = torch.zeros(525, 14, dtype=torch.float)
                    input_tensor[0:time] = unit_input[0:time]
                    unit_RUL = unit_life - time
                    input_signal.append(input_tensor)
                    RUL.append(unit_RUL)
                    label.append(unit_label)
            else:
                for time in range(3, unit_life):
                    input_tensor = unit_input[0:time]
                    unit_RUL = unit_life - time
                    input_signal.append(input_tensor)
                    RUL.append(unit_RUL)
                    label.append(unit_label)

        self.RUL = np.array(RUL)
        self.input_signal = input_signal
        self.all_labels = label

    def __len__(self):
        return len(self.RUL)

    def __getitem__(self, idx):
        data = {
            "input": self.input_signal[idx],
            "RUL": torch.tensor(self.RUL[idx], dtype=torch.int64),
            "mode":self.all_labels[idx]
        }

        return data

In [184]:
CFG.ae_input_layer=6
CFG.ae_hidden_layer=8

In [185]:
train_dataset_expend=AircraftDataset_expend(df_train,train_label,False)

In [186]:
train_dataset_expend[256]

{'input': tensor([[ 1.0474,  1.2005,  0.2477, -0.2516,  0.2673, -0.4300],
         [ 0.6587,  0.6947,  0.4586, -0.1370, -0.0272, -0.2752],
         [ 0.6080,  1.0038,  0.0554, -0.2484,  0.6095, -0.2855]]),
 'RUL': tensor(250),
 'mode': tensor([0., 1.])}

In [187]:
 def my_collate(batch):
 # batch contains a list of tuples of structure (sequence, target)
    data = [item["input"] for item in batch]
    data = pack_sequence(data, enforce_sorted=False)
    targets = [item["mode"] for item in batch]
    res={
        "input":data,
        "mode":torch.vstack(targets)
        
    }
    
    return res

In [188]:
train_data_expend_loader = DataLoader(train_dataset_expend,
                                      batch_size=CFG.ae_batch_size,
                                      shuffle=True,
                                      drop_last=True,
                                      collate_fn=my_collate)

In [242]:
class LSTMAutoEncoder(nn.Module):
    def __init__(self,input_layer=CFG.ae_input_layer,hidden_layer=CFG.ae_hidden_layer,ae_num_class=2):
        super().__init__()
        self.input_layer = input_layer
        self.hidden_layer = hidden_layer
        self.num_class= ae_num_class
        self.encoder1=nn.LSTM(self.input_layer, self.hidden_layer, batch_first=True)
        self.encoder2=nn.LSTM(self.hidden_layer,self.hidden_layer*2,batch_first=True)
        self.decoder1=nn.LSTM(self.hidden_layer*2,self.hidden_layer,batch_first=True)
        self.decoder2=nn.LSTM(self.hidden_layer,self.input_layer,batch_first=True)
        self.logits = nn.Sequential(
            nn.Linear(self.hidden_layer*2, self.hidden_layer),
            nn.ReLU(),
            nn.Linear(self.hidden_layer, self.num_class),
            nn.LogSoftmax(dim=-1),
        )


    def forward(self, x):
        x, (_,_) = self.encoder1(x)
        x, (h0,_) = self.encoder2(x)
        x,(_,_)= self.decoder1(x)
        x,(_,_)= self.decoder2(x)
        
        label= self.logits(h0)
                                                  

        return x,label

In [243]:
aemodel=LSTMAutoEncoder()

In [244]:
for train_data in train_data_expend_loader:
    None

In [245]:
aemodel(train_data["input"])

(PackedSequence(data=tensor([[-0.0098,  0.0122,  0.1225, -0.0200,  0.0868, -0.0897],
         [-0.0098,  0.0122,  0.1225, -0.0200,  0.0868, -0.0897],
         [-0.0098,  0.0122,  0.1225, -0.0200,  0.0868, -0.0897],
         ...,
         [-0.0284,  0.0033,  0.2189, -0.0309,  0.1651, -0.2215],
         [-0.0283,  0.0033,  0.2189, -0.0308,  0.1651, -0.2215],
         [-0.0283,  0.0034,  0.2190, -0.0308,  0.1650, -0.2215]],
        grad_fn=<CatBackward0>), batch_sizes=tensor([512, 512, 512, 510, 508, 506, 506, 504, 502, 499, 497, 494, 490, 488,
         486, 483, 482, 478, 473, 471, 468, 466, 462, 461, 455, 453, 452, 450,
         445, 441, 440, 437, 435, 434, 430, 426, 426, 423, 422, 419, 418, 417,
         416, 412, 411, 409, 404, 404, 403, 400, 397, 396, 394, 392, 388, 387,
         384, 382, 382, 380, 380, 380, 377, 374, 372, 370, 369, 366, 363, 362,
         362, 360, 358, 357, 355, 353, 352, 348, 347, 346, 345, 343, 341, 339,
         336, 333, 330, 328, 327, 325, 324, 323, 321, 320

In [246]:
x=train_data["input"].data
x_=aemodel(train_data["input"])[0].data

In [247]:
class_true=train_data["mode"]
class_pred=aemodel(train_data["input"])[1].squeeze(0)

In [248]:
class_pred,class_true

(tensor([[-0.7674, -0.6240],
         [-0.7672, -0.6242],
         [-0.7646, -0.6265],
         ...,
         [-0.7699, -0.6218],
         [-0.7632, -0.6277],
         [-0.7668, -0.6246]], grad_fn=<SqueezeBackward1>),
 tensor([[0., 1.],
         [1., 0.],
         [0., 1.],
         ...,
         [1., 0.],
         [1., 0.],
         [0., 1.]]))

In [221]:
b = nn.MSELoss()(x,x_)
a = nn.CrossEntropyLoss()(class_pred, class_true)

In [222]:
b,a

(tensor(0.5862, grad_fn=<MseLossBackward0>),
 tensor(0.6929, grad_fn=<DivBackward1>))

In [255]:
-torch.sum(class_pred*class_true)/len(class_pred)

tensor(0.6900, grad_fn=<DivBackward0>)

In [256]:
nn.MSELoss()(x,x_)-torch.sum(class_pred*class_true)/len(class_pred)

tensor(1.1342, grad_fn=<SubBackward0>)